# Using TensorFlow

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()
warnings.simplefilter(action='ignore', category=ImportWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=ResourceWarning)

In [ ]:
import numpy as np
import tensorflow as tf

## Check Hardware Availability

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## CPU vs GPU

In [ ]:
%%time
m = 10000
n = 10000
p = 10000

with tf.device('/cpu:0'):
    a = tf.random_normal([m, n], mean=0, stddev=1)
    b = tf.random_normal([n, p], mean=0, stddev=1)
    c = tf.matmul(a, b)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
res = sess.run(c)
print(res.shape, res.dtype)

In [ ]:
%%time
m = 10000
n = 10000
p = 10000

with tf.device('/gpu:0'):
    a = tf.random_normal([m, n], mean=0, stddev=1)
    b = tf.random_normal([n, p], mean=0, stddev=1)
    c = tf.matmul(a, b)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
res = sess.run(c)
print(res.shape, res.dtype)

## Basic computations in TensorFlow

### Starting a session

In [ ]:
tf.InteractiveSession()

### Creation of tensors

In [ ]:
a = tf.zeros((3,5))
a.eval()

In [ ]:
a.dtype, a.shape

In [ ]:
b = tf.ones((3,5))
b.eval()

In [ ]:
c = tf.fill((3,5), value=23.0)
c.eval()

In [ ]:
d = tf.eye(5)
d.eval()

In [ ]:
e = tf.diag(tf.range(1.0,6.0))
e.eval()

### Generating random numbers

In [ ]:
x = tf.random_normal((5,3))
x.eval()

In [ ]:
y = tf.random_uniform((5,3))
y.eval()

In [ ]:
z = tf.random_poisson(5, (5,3))
z.eval()

## Opearations and graphs

#### The object stored in a variable is not its value.

In [ ]:
z

In [ ]:
z.eval()

In [ ]:
z.eval()

#### Element-wise operations

In [ ]:
op1 = (a + b) * c
op1.eval()

In [ ]:
op2 = tf.multiply(tf.add(a, b), c)
op2.eval()

#### Matrix operations

In [ ]:
op3 = tf.matmul(c, tf.transpose(c))
op3.eval()

In [ ]:
c.eval()

In [ ]:
tf.reshape(c, (5,-1)).eval()

### Broadcasting

In [ ]:
(c - 22).eval()

In [ ]:
(c + tf.ones((3, 1))).eval()

In [ ]:
(c * tf.constant(2.0, shape=(1, 5))).eval()

### Explicit Casts

In [ ]:
x1 = tf.eye(3)

In [ ]:
x2 = tf.eye(3, dtype=tf.int32)

In [ ]:
try:
    x1 + x2
except ValueError as e:
    print(e)

In [ ]:
x2 = tf.cast(x2, dtype=tf.float32)

In [ ]:
(x1 + x2).eval()

## Reductions

In [ ]:
x = tf.random_normal((10,1), 0, 1)
x.eval()

#### Mean

In [ ]:
tf.reduce_mean(x).eval()

#### Sum of squares

In [ ]:
tf.reduce_sum(tf.square(x)).eval()

## Data Flow Graphs 

In [ ]:
tf.get_default_graph()

Run if necessary
```bash
! pip install graphviz
```

Source for graph drawing code: [Visualizing TensorFlow Graphs in Jupyter Notebooks](https://blog.jakuba.net/2017/05/30/tensorflow-visualization.html)

In [ ]:
from graphviz import Digraph

In [ ]:
def tf_to_dot(graph):
    dot = Digraph()

    for n in g.as_graph_def().node:
        dot.node(n.name, label=n.name)

        for i in n.input:
            dot.edge(i, n.name)
            
    return dot

In [ ]:
g = tf.Graph()

with g.as_default():
    a = tf.constant(1, name='a')
    b = tf.constant(2, name='b')
    c = tf.constant(3, name='c')
    ans = tf.multiply(tf.add(a, b), c)
    
tf_to_dot(g)

## Constants, Variables and Placeholders

### Constants

Not much to explain here. Use constants to store invariant values, for example - fixed hyperparameters.

In [ ]:
a = tf.constant(3.0)
a.eval()

In [ ]:
b = tf.ones(3)
b.eval()

### Variables

In [ ]:
c = tf.Variable(tf.ones(3))

In [ ]:
try:
    c.eval()
except tf.errors.FailedPreconditionError as e:
    print(e)

In [ ]:
with tf.Session() as s1:
    s1.run(tf.global_variables_initializer())
    print(s1.run(c))

In [ ]:
with tf.Session() as s1:
    s1.run(c.assign(5*b))
    print(s1.run(c))

Variables are updated on each pass of the data flow graph. They are used to store parameters that must be updated, for example, weights and biases of a network.

In [ ]:
g = tf.Graph()

with g.as_default():
    w = tf.Variable(0, name='w')
    
tf_to_dot(g)

### Placeholders

Placeholders are used to feed in data when the data flow graph is run.

In [ ]:
x = tf.placeholder(tf.float32, shape=(3,3))
y = tf.matmul(x, x)

data = np.random.rand(3, 3)

with tf.Session() as s1:
    print(s1.run(y, feed_dict={x: data})) 